In [6]:
import inspect
import os
import pathlib
import cv2


HOME = os.environ.get('HOME_PATH')
VIDEO_PATH = os.path.join(HOME, "pipeline", "1. videos")
FRAME_PATH = os.path.join(HOME, "pipeline", "2. frames")
FRAME_PATH_UNIQUE = os.path.join(HOME, "pipeline", "2. frames_unique")
FRAME_PATH_DROP = os.path.join(HOME, "pipeline", "2. frames_duplicate_dropped")
FRAME_PATH_DISTINCT = os.path.join(HOME, "pipeline", "2. frames_distinct")
FRAME_PATH_SIMILAR_DROP = os.path.join(HOME, "pipeline", "2. frames_similar_drop")
WINDOW_PATH = os.path.join(HOME, "pipeline", "3. windows")
WINDOW_PATH_UNIQUE = os.path.join(HOME, "pipeline", "3. windows_unique")
WINDOW_PATH_DROP = os.path.join(HOME, "pipeline", "3. windows_dropped")
PREDICTED_VEHICLE_PATH = os.path.join(HOME, "pipeline", "4. predicted_vehicles")
PREDICTED_NO_VEHICLE_PATH = os.path.join(HOME, "pipeline", "4. predicted_no_vehicles")

# Classes

In [7]:
# Image processing methods
# Attempt to abstract to allow different impelementation (cv2, pillow, etc)
# class ImageManager:
#     def __init__(self, name):    
#         self.name = name
 
#     def getImage(self):    
#         raise NotImplementedError("Subclass must implement abstract method")
        
#     def resize(self, (width, height)):    
#         raise NotImplementedError("Subclass must implement abstract method")
    
#     def dhash(self):
#         raise NotImplementedError("Subclass must implement abstract method")
    
# class PillowImageManager(ImageManager):
#     def resize(self, (width, height)): 
        
# class Cv2ImageManager(ImageManager):
#     def resize(self, (width, height)): 
    
# def resizeImagePillow(image, width):
#     height = round(width * image.size[1]/image.size[0])
#     image.resize((width,height))
    

# Split Frames

In [8]:
import inspect
import os
import pathlib
import subprocess 
import time

FFMPEG_BINARY = "C:\\Program Files (x86)\\ffmpeg\\ffmpeg-4.1.1-win64-static\\bin\\ffmpeg.exe"

In [9]:
def splitFramesFromVideo(src, dest):
    print("Getting frames from ", src)
    
    videoName = os.path.split(src)[1]
    pathlib.Path(dest).mkdir(parents=True, exist_ok=True) 

    command = [ FFMPEG_BINARY,
                '-i', src,
                '-r', '1/1',
                dest + '/' + videoName + '.frame-%03d.png']
    proc = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, bufsize=10**8)
    proc.stderr.read()

In [10]:
def splitFramesFromVideoJob(src, dest):
          
    print("Checking ", src)
    
    for filename in glob.iglob(os.path.join(src, '**/*.mp4'), recursive=True):   

        filepath =  os.path.join(src, filename)
        splitFramesFromVideo(filepath, dest)

        # remove file once processed
        os.remove(filepath)


# Remove Duplicate Images

In [11]:
import shutil
import glob
from PIL import Image
import numpy as np
from skimage import io
import scipy

In [12]:
def dhash(image, hash_size = 8):
    # Grayscale and shrink the image.
    
    image = image.convert('LA')
    image = image.resize((hash_size+1,hash_size))
   
    pixels = [pixel for pixel, alpha in image.getdata()]
    pixels = np.array(pixels).reshape((8, 9))
    
    #pixels = image
    # Compare adjacent pixels.
    difference = []
    for row in range(hash_size):
        for col in range(hash_size):
            pixel_left = pixels[row, col]
            pixel_right = pixels[row, col + 1]
            difference.append(pixel_left > pixel_right)
    # Convert the binary array to a hexadecimal string.
    decimal_value = 0
    hex_string = []
    
    for index, value in enumerate(difference):
        if value:
            decimal_value += 2**(index % 8)
        if (index % 8) == 7:
            hex_string.append(hex(decimal_value)[2:].rjust(2, '0'))
            decimal_value = 0
    return ''.join(hex_string)


def getImage(path):
    image = cv2.imread(path)
#     image = image.reshape((50, 100,3))
#     image = Image.open(path)
    return image

def get_similar_score(img_a, img_b):
    
    SIMILAR_THRESHOLD = 10
    
    image_pixel_count = np.prod(img_a.shape)
    image_chanel_pixel_count = image_pixel_count / 3
    
    if img_a.shape == img_b.shape :
        
        difference = cv2.subtract(img_a, img_b)
        
        # Treat small differences as zero
        difference[difference < SIMILAR_THRESHOLD] = 0
        b = difference[:,:,0] 
        g = difference[:,:,1] 
        r = difference[:,:,2] 
        
        return np.sum(difference) / image_pixel_count
    
    else:
        raise TypeError("Images must be same size.") 

# def get_sub_image(image):
#     left = 480
#     width = 1320
#     top = 400
#     height = 900

#     return image[top:height,left:width,:] 

def get_mse(image_a, image_b):
    sub_a = get_sub_image(image_a)
    sub_b = get_sub_image(image_b)
    err = np.sum((sub_a.astype("float") - sub_b.astype("float")) ** 2)
    image_pixel_count = np.prod(sub_a.shape)
    err /= float(image_pixel_count)
    return err

CHANGE_RATE = 0.065
TUMBNAIL_SIZE = 30
SUB_IMAGE_CENTER = (int(1920 / 2), int(1080 / 2) + 100)
SUB_IMAGE_RADIUS = 250

def get_sub_image(image):
    left = SUB_IMAGE_CENTER[0] - SUB_IMAGE_RADIUS
    width = left + SUB_IMAGE_RADIUS * 2
    top = SUB_IMAGE_CENTER[1] - SUB_IMAGE_RADIUS
    height = top + SUB_IMAGE_RADIUS * 2

    return image[top:height,left:width,:] 
    
def img_gray(image):    
    return np.average(image, weights=[0.299, 0.587, 0.114], axis=2)

def resize_image(image, height=TUMBNAIL_SIZE, width=TUMBNAIL_SIZE):
    row_res = cv2.resize(image, (height, width), interpolation = cv2.INTER_AREA).flatten()
    col_res = cv2.resize(image, (height, width), interpolation = cv2.INTER_AREA).flatten('F')
    return row_res, col_res

def intensity_diff(row_res, col_res):
    # Checks if neighbor pixel is greater or less.
    difference_row = np.diff(row_res)
    difference_col = np.diff(col_res)
    difference_row = difference_row > 0
    difference_col = difference_col > 0
    return np.vstack((difference_row, difference_col)).flatten()

def image_hash(image, height = TUMBNAIL_SIZE, width = TUMBNAIL_SIZE):
    sub_image = get_sub_image(image)
    gray = img_gray(sub_image)    
    row_res, col_res = resize_image(sub_image, height, width)
    image_hash = intensity_diff(row_res, col_res)

    return image_hash
    
def hamming_distance(image, image2):
    score = scipy.spatial.distance.hamming(image, image2)
    return score

In [31]:
def binary_threshold(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.medianBlur(gray,5)
    _,th2 = cv2.threshold(gray,124, 255,cv2.THRESH_BINARY)
    th2 = cv2.cvtColor(th2, cv2.COLOR_GRAY2BGR)
    return th2

In [32]:
def filter_image(image):
    return binary_threshold(image)

In [23]:
def removeDuplicatesJob(src, dest, dropped_dest):
    
    pathlib.Path(dest).mkdir(parents=True, exist_ok=True) 
    pathlib.Path(dropped_dest).mkdir(parents=True, exist_ok=True) 
    
    print("Check for duplicates in ", src)
    # Add image hash to dictionary
    imageDictionary = dict()

    i = 0
    for filename in glob.iglob(os.path.join(src, '**/*.png'), recursive=True):
        
        img = getImage(filename)
        idx = dhash(img)
        if idx not in imageDictionary:
            imageDictionary[idx] = filename

        else: 
            shutil.move(filename, dropped_dest)
            i += 1
#         imageDictionary[idx].append(filename)
       
    print("removed " +str(i) + " files")
    
    # move remaining to dest
    i = 0
    for filename in glob.iglob(os.path.join(src, '**/*.png'), recursive=True):
        shutil.move(filename, dest)
        i += 1

    print("kept " +str(i) + " files")
    

def remove_similar_images_job_hamming(src, dest, dropped_dest):
    print("Check for distinct images - hamming", src)
    


    # Create folder if not already exists
    pathlib.Path(dest).mkdir(parents=True, exist_ok=True) 
    pathlib.Path(dropped_dest).mkdir(parents=True, exist_ok=True) 

    distinct_images = []
    similar_images = []
    
    i = 0
    j = 0

    
    file_names = list(glob.iglob(os.path.join(src, '**/*.png'), recursive=True))
    
    # get first  
    base_filename = file_names[0]
    base_image = getImage(base_filename)
    base_image = filter_image(base_image) # This is our filter image    
    base_image_hash = image_hash(base_image)
    distinct_images.append(base_filename)
    
    for file_name in file_names[1:]:
        compare_image = getImage(file_name)    
        compare_image = filter_image(compare_image)
        compare_image_hash = image_hash(compare_image)
        hd = hamming_distance(base_image_hash, compare_image_hash)  
     
        if(hd > CHANGE_RATE ):
            base_filename = file_name
            base_image_hash = compare_image_hash 
            distinct_images.append(file_name) 
        else:
            similar_images.append(file_name)
            
   
    #-------------
    for file_name in distinct_images:
        dest_filename = os.path.join(dest, os.path.basename(file_name))
        shutil.move(file_name, dest_filename)
        
    for file_name in similar_images:
        dest_filename = os.path.join(dropped_dest, os.path.basename(file_name))
        shutil.move(file_name,dest_filename)
        
    print("Kept " + str(len(distinct_images)) + " images. Dropped " + str(len(similar_images)))
    
    
# def remove_similar_images_job_mse(src, dest, dropped_dest):
    
#     print("Check for distinct images ", src)
    
#     # Create folder if not already exists
#     pathlib.Path(dest).mkdir(parents=True, exist_ok=True) 
#     pathlib.Path(dropped_dest).mkdir(parents=True, exist_ok=True) 
    
#     THRESHOLD_RATE = 1000
    
#     distinct_images = []
#     similar_images = []
    
#     i = 0
#     j = 0
#     file_generator = glob.iglob(os.path.join(src, '**/*.png'), recursive=True)
    
#     # Treat first item as unique
#     base_filename = next(file_generator)
#     base_image = getImage(base_filename)
#     distinct_images.append(base_filename)
#     i += 1
        
#     for filename in file_generator:
        
#         compare_image = getImage(filename)
        
#         mse = get_mse(base_image, compare_image )
#         print(mse, base_filename, " with ", filename)
        
#         if(mse < THRESHOLD_RATE):
#             print("Add to similar")
#             similar_images.append(filename)
#         else:
#             print("Add to distinct")
#             base_filename = filename
#             base_image = getImage(base_filename)
#             distinct_images.append(filename)
            
#     for filename in distinct_images:
#         shutil.move(filename,dest)
        
#     for filename in similar_images:
#         shutil.move(filename,dropped_dest)
        
#     print("Kept " + str(len(distinct_images)) + " images. Dropped " + str(len(similar_images)))

In [ ]:
def test_hamming_score(src):
    
    THRESHOLD_RATE = 1000
    
    distinct_images = []
    similar_images = []
    
    i = 0
    j = 0
    file_names = list(glob.iglob(os.path.join(src, '**/*.png'), recursive=True))
    
    # Treat first item as unique
    base_filename = file_names[0]
    base_image = getImage(base_filename)
    base_image = filter_image(base_image) # This is our filter image    
    base_image_hash = image_hash(base_image)
    distinct_images.append(base_filename)
    i += 1
        
    for file_name in file_names[1:]:
        compare_image = getImage(file_name)    
        compare_image = filter_image(compare_image)
        compare_image_hash = image_hash(compare_image)
        hd = hamming_distance(base_image_hash, compare_image_hash)        
        print(hd, base_filename.split('\\')[-1], " with ", file_name.split('\\')[-1])

test_hamming_score(FRAME_PATH)

In [24]:
# from sklearn.cluster import MiniBatchKMeans
# def test_color_quantization(src):
    
#     N_CLUSTERS = 3
    
#     file_names = list(glob.iglob(os.path.join(src, '**/*.png'), recursive=True))
    
#     # Treat first item as unique
#     base_filename = file_names[0]
#     image = getImage(base_filename)
#     plt.imshow(image)
    
#     (h, w) = image.shape[:2]
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
#     image = image.reshape((image.shape[0] * image.shape[1], 3))
#     clt = MiniBatchKMeans(N_CLUSTERS)
#     labels = clt.fit_predict(image)
#     quant = clt.cluster_centers_.astype("uint8")[labels]
    
#     quant = quant.reshape((h, w, 3))
#     image = image.reshape((h, w, 3))
    
#     quant = cv2.cvtColor(quant, cv2.COLOR_LAB2BGR)
#     image = cv2.cvtColor(image, cv2.COLOR_LAB2BGR)
    
#     print("Hello")
    
#     cv2.imshow("image", np.hstack([image, quant]))
    
#     plt.imshow(quant)
    
# test_color_quantization(FRAME_PATH)

In [25]:
# import argparse
# def test_median_filter(src):
#     file_names = list(glob.iglob(os.path.join(src, '**/*.png'), recursive=True))
    
#     # Treat first item as unique
#     base_filename = file_names[0]
#     image = getImage(base_filename)
# #     plt.imshow(image)
    
    
#     # create the argument parser and parse the arguments
# #     ap = argparse.ArgumentParser()
# #     ap.add_argument('-i', '--image', required = True, help = base_filename)
# #     args = vars(ap.parse_args())
    
#     processed_image = cv2.medianBlur(image, 9)
#     plt.imshow(processed_image)

# test_median_filter(FRAME_PATH)

In [26]:
          
# def test_errosion(src):
    
    
          
#     file_names = list(glob.iglob(os.path.join(src, '**/*.png'), recursive=True))
    
#     # Treat first item as unique
#     base_filename = file_names[0]
#     image = getImage(base_filename)
          
#     kernel = np.ones((9,9),np.uint8)
#     erosion = cv2.erode(image,kernel,iterations = 1)    
    
#     plt.imshow(erosion)
    
# test_errosion(FRAME_PATH)

In [27]:
# from random import shuffle


# HOME = os.environ.get('HOME_PATH')
# LOCAL_FRAME_PATH = os.path.join(HOME, "local", "pipeline_2_frames")
# LOCAL_FRAME_PATH_DISTINCT = os.path.join(HOME, "pipeline", "2_frames_distinct")
# LOCAL_FRAME_PATH_SIMILAR_DROP = os.path.join(HOME, "pipeline", "2_frames_similar_drop")
# CONTAINER_FRAMES = 'pipeline-2-frames'
# CONTAINER_FRAMES_DISTINCT = 'pipeline-2-frames-distinct'
# CONTAINER_SIMILAR_DROP = 'pipeline-2-frames-similar-drop'


# def doit():
    
#     # Create Local folders
#     pathlib.Path(LOCAL_FRAME_PATH).mkdir(parents=True, exist_ok=True)
    
#     blobs = []
    
#     file_generator = glob.iglob(os.path.join(FRAME_PATH, '**/*.png'), recursive=True)
#     for filename in file_generator:
#         blobs.append(filename.split('\\')[-1])
        
#     shuffle(blobs)
#     print(blobs[:10] )
    
#     blobs.sort()
#     print(blobs[:10])
    
#     # Get Local file
#     for blob in blobs:
#         dest_filename = os.path.join(VIDEO_PATH, blob.name)
#         blob_service.get_blob_to_path(
#             CONTAINER_VIDEOS,
#             blob.name,
#             dest_filename
#             )
    
# doit()

# Create Windows

In [11]:
import imutils

In [12]:
def pyramid(image, scale=1.5, minSize=(100, 50)):
    # yield the original image
    pixels = np.array(image)
    
    yield pixels
 
    # keep looping over the pyramid
    while True:
              
        # compute the new dimensions of the image and resize it
        w = int(pixels.shape[1] / scale)
        image = imutils.resize(pixels, width=w)
 
        # if the resized image does not meet the supplied minimum
        # size, then stop constructing the pyramid
        if image.shape[0] < minSize[1] or image.shape[1] < minSize[0]:
            break
 
        # yield the next image in the pyramid
        yield pixels
        
def buildImageName(src, x, y, scale=1):
    parts =  src.split('\\')
    videoFile = parts[-1]
    videoFileName = videoFile.split('.')[0]
    
    frameNo = parts[-1].split('.')[-2].split('-')[1]
    
    return videoFileName + '-' + frameNo + '-' + str(scale) + '-' + str(x) + '-' + str(y) + '.jpg'
        
def sliding_window(pixels, stepSize=20, windowSize=(100,50)):
    
    winW = windowSize[0]
    winH = windowSize[1]
    
    # slide a window across the image
    for y in range(0, pixels.shape[0], stepSize):
        for x in range(0, pixels.shape[1], stepSize):
            
            window = pixels[y:y+windowSize[1],x:x+windowSize[0]]
            
            # if the window does not meet our desired window size, ignore it
            if window.shape[0] != winH or window.shape[1] != winW:
                continue
                
            # yield the current window
            yield (x, y, window)
        
def splitWindowsFromFramesJob(src, dest, scale=1):
    
    paths = []
    
    print("Spltting Windows for ", src)
    
    # create dest if it doesnt' exist
    pathlib.Path(dest).mkdir(parents=True, exist_ok=True) 
    
    i = 0
    for filename in glob.iglob(os.path.join(src, '**/*.png'), recursive=True):    
        
        print("Spltting Windows for ", filename)
                
        image = getImage(filename)
        
        for (i, resized) in enumerate(pyramid(image, scale=scale)):

            for x, y, window in sliding_window(resized):    

                destImageName = buildImageName(filename, x, y, scale=i+1)   
                
                w=Image.fromarray(window,mode='RGB')
                fullpath =os.path.join(dest, destImageName) 
                w.save(fullpath)
                paths.append(fullpath)
                
        if i > 2:
            break
    
    return paths


# Score Image

In [13]:
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient  
import json
import requests
import numpy as np
from PIL import Image
import matplotlib.patches as patches

PREDICTION_ENDPOINT = os.environ.get("PREDICTION_ENDPOINT") 
PREDICTION_KEY =  os.environ.get("PREDICTION_KEY") 
PREDICITON_RESOURCE_ID =  os.environ.get("PREDICITON_RESOURCE_ID") 

PROJECT_ID =  os.environ.get("PROJECT_ID") 
PUBLISH_ITERATION_NAME =  os.environ.get("PUBLISH_ITERATION_NAME") 


url =  '{Endpoint}/{projectId}/detect/iterations/{publishedName}/image'
path_format_arguments = {
    'Endpoint': PREDICTION_ENDPOINT,
    'projectId': PROJECT_ID,
    'publishedName': PUBLISH_ITERATION_NAME
}
PREDICTION_URL = url.format(**path_format_arguments)

HEADER_PARAMETERS = {
    'accept' : 'application/json',
    'content-type' : 'application/octet-stream',
    'Prediction-Key' : PREDICTION_KEY
}

In [14]:
def print_score_prediction(prediction):
    bounding_box =  prediction['boundingBox']
    pre = {
        'probability': prediction['probability'],
        'tagName' : prediction['tagName'],
        'bb_left' : bounding_box['left'],
        'bb_top' : bounding_box['top'],
        'bb_width' : bounding_box['width'],
        'bb_height' : bounding_box['height']                        
    }
    return pre
    print("{tagName} {probability} - bbox.left = {bb_left}, bbox.top = {bb_top}, bbox.width = {bb_width}, bbox.height = {bb_height}".format(**pre))
    
def score_image(src):
    
    with open(src, mode="rb") as image:
    
        response  = requests.post(url=PREDICTION_URL,
                            data=image,
                            headers=HEADER_PARAMETERS)

        if response.status_code == 200:
            results = json.loads(response.content.decode('utf-8'))

            return results['predictions'] 
            
        else:
            print(response)
            raise Exception('Failed to get score.', response.status_code)

def get_top_prediction(predictions):
    
    top_score = 0
    top_prediction = {}
    
    for prediction in predictions:
                
        if(prediction['probability'] > top_score ):
            top_score = prediction['probability'] 
            top_prediction = prediction
            
    
    return top_prediction
        
def save_prediction(filename, predictions, dest):
    meta_filename = filename + '.annotation.json'
    
    full_path = os.path.join(dest, meta_filename)
    
    with open(full_path, 'w') as outfile:
        json.dump(predictions, outfile)


def test_score_image_save(filename, dest):
    predictions = score_image(filename)
    save_prediction(filename, predictions, dest)
    
def score_image_from_frames_job(src, predicted_vechicle_dest, predicted_no_vechicle_dest):
    
    score_threshold = 0.8
    i = 0
    paths = []
    
    
    print("Scoring images in ", src)
    
    # create dest if it doesnt' exist
    pathlib.Path(predicted_vechicle_dest).mkdir(parents=True, exist_ok=True) 
    pathlib.Path(predicted_no_vechicle_dest).mkdir(parents=True, exist_ok=True) 
    
    i = 0
    j = 0
    for filename in glob.iglob(os.path.join(src, '**/*.png'), recursive=True):  
        predictions = score_image(filename)
        save_prediction(filename, predictions, src)
        prediction = get_top_prediction(predictions)
        
        if(prediction):
            if(prediction['probability'] > score_threshold ):
               
                dst_filename = os.path.join(predicted_vechicle_dest, os.path.basename(filename))
                shutil.move(filename, dst_filename)
                dst_filename_annon = os.path.join(predicted_vechicle_dest, os.path.basename(filename + '.annotation.json'))
                shutil.move(filename + '.annotation.json', dst_filename_annon)
                
                i += 1
            else :                
                dst_filename = os.path.join(predicted_no_vechicle_dest, os.path.basename(filename))
                shutil.move(filename, dst_filename)
                dst_filename_annon = os.path.join(predicted_no_vechicle_dest, os.path.basename(filename + '.annotation.json'))
                shutil.move(filename + '.annotation.json', dst_filename_annon)
                j += 1
        else:
            dst_filename = os.path.join(predicted_no_vechicle_dest, os.path.basename(filename))
            shutil.move(filename, dst_filename)
            dst_filename_annon = os.path.join(predicted_no_vechicle_dest, os.path.basename(filename + '.annotation.json'))
            shutil.move(filename + '.annotation.json', dst_filename_annon)
            j += 1
                
                
    print("Labeled " +str(i) + " files")
    print("Dropped " +str(j) + " files")
     

In [15]:
# test_image_path = os.path.join(HOME_PATH, 'frames\\20180326_071212_NF.mp4\\frame-003.png')
# test_image_save_path = os.path.join(HOME_PATH, 'frames\\20180326_071212_NF.mp4\\')
# test_score_image_save(test_image_path, test_image_save_path)

In [16]:
import matplotlib.pyplot as plt
%matplotlib inline

def show_predictions(image, predictions):
    
    im = np.array(Image.open(test_image_path), dtype=np.uint8)

    plt.figure(num=None, figsize=(15 ,15), dpi=80, facecolor='w', edgecolor='k')
    plt.imshow( im )
    
    for prediction in predictions:
        
        bbox_left = 1920 * prediction['boundingBox']['left']
        bbox_top = 1080 * prediction['boundingBox']['top']
        bbox_width = 1920 * prediction['boundingBox']['width']
        bbox_height = 1080 * prediction['boundingBox']['height']
       
        ax = plt.gca()
        rect = patches.Rectangle((bbox_left,bbox_top),bbox_width,bbox_height,linewidth=1,edgecolor='r',facecolor='none')
        ax.add_patch(rect)

In [ ]:
predictions = score_image(test_image_path)
print(predictions)

In [ ]:
# preditions = json.loads('[{"probability": 0.4296951, "tagId": "991ba077-b9b4-4538-a6e5-8eab7e5daf1e", "tagName": "license", "boundingBox": {"left": 0.6178843, "top": 0.6097945, "width": 0.06222242, "height": 0.0467823148}}, {"probability": 0.0109245218, "tagId": "991ba077-b9b4-4538-a6e5-8eab7e5daf1e", "tagName": "license", "boundingBox": {"left": 0.1954996, "top": 0.946872056, "width": 0.08564204, "height": 0.05311793}}, {"probability": 0.242702723, "tagId": "71adc55b-fa59-4cc0-8d43-863193b45ecc", "tagName": "vehicle", "boundingBox": {"left": 0.770755768, "top": 0.5213496, "width": 0.08635795, "height": 0.110935867}}, {"probability": 0.07627616, "tagId": "71adc55b-fa59-4cc0-8d43-863193b45ecc", "tagName": "vehicle", "boundingBox": {"left": 0.847536147, "top": 0.5082251, "width": 0.0618121028, "height": 0.103719473}}, {"probability": 0.9402402, "tagId": "71adc55b-fa59-4cc0-8d43-863193b45ecc", "tagName": "vehicle", "boundingBox": {"left": 0.5640773, "top": 0.461323023, "width": 0.211154938, "height": 0.302533269}}]')
show_predictions(test_image_path, preditions)

# Pipeline Jobs

In [ ]:
splitFramesFromVideoJob(VIDEO_PATH, FRAME_PATH)

In [29]:
#removeDuplicatesJob(FRAME_PATH, FRAME_PATH_UNIQUE, FRAME_PATH_DROP )

In [ ]:
remove_similar_images_job_hamming(FRAME_PATH, FRAME_PATH_DISTINCT, FRAME_PATH_SIMILAR_DROP)

In [21]:
# shutil.move(os.path.join(HOME_PATH, 'pipeline\\2. frames\\20190330_165033_EF.mp4.frame-014.png'), os.path.join(HOME_PATH, 'pipeline\\2. frames_distinct\\20190330_165033_EF.mp4.frame-014.png'))

In [ ]:
score_image_from_frames_job(FRAME_PATH_DISTINCT, PREDICTED_VEHICLE_PATH, PREDICTED_NO_VEHICLE_PATH)

In [ ]:
# All at once
# splitFramesFromVideoJob(VIDEO_PATH, FRAME_PATH)
# removeDuplicatesJob(FRAME_PATH, FRAME_PATH_UNIQUE, FRAME_PATH_DROP )
# score_image_from_frames_job(FRAME_PATH_UNIQUE, PREDICTED_VEHICLE_PATH, PREDICTED_NO_VEHICLE_PATH)

In [ ]:
# paths = splitWindowsFromFramesJob(FRAME_PATH_UNIQUE, WINDOW_PATH)

In [ ]:

# x_image = getImage(os.path.join(HOME_PATH, 'pipeline\\2. frames_unique\\20180326_071212_NF.mp4-frame-001.png'))

In [ ]:
# x_image.size


In [ ]:
# x_pixels = np.array(x_image)

In [ ]:
# print(x_pixels.shape)
# x_width = 64

# x_height = round(x_width * x_image.size[1]/x_image.size[0])

# print(x_height)

In [ ]:
# x_src = os.path.join(HOME_PATH, 'pipeline\\2. frames_unique\\20180326_071212_NF.mp4.frame-001.png')
# x_parts =  x_src.split('\\')
# print(x_parts)

# x_videoFile = x_parts[-1].split(".")
# print(x_videoFile)

# x_frameNo = x_parts[-1].split(".")[-2]
# print(x_frameNo)

# # want the format 20180326_071212_NF-006-1-0-0

# buildImageName(x_src,10,10)

In [29]:
import sys
print(sys.version)

3.6.8 |Anaconda, Inc.| (default, Feb 21 2019, 18:30:04) [MSC v.1916 64 bit (AMD64)]
